In [7]:
from sqlalchemy import create_engine
import pandas as pd

In [8]:
# Datos de conexión - reemplaza con los tuyos
host = 'sql301.infinityfree.com'    # Reemplaza con el host de tu base de datos
database = 'if0_37643605_sp500_historico'      # Reemplaza con el nombre de tu base de datos
user = 'if0_37643605'                   # Reemplaza con tu nombre de usuario
password = 'zZcoMVyPxCO5y'              # Reemplaza con tu contraseña

# Crear la conexión usando SQLAlchemy
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')


In [9]:
### 1. Poblar la Tabla `empresas` ###

# Cargar el CSV de empresas
df_empresas = pd.read_csv("../Streamlit/infoSP500.csv")  # Reemplaza con la ruta al archivo real

# Seleccionar y renombrar columnas para que coincidan con la tabla `empresas`
df_empresas = df_empresas[['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry', 'Date added']]
df_empresas.columns = ['ticker', 'nombre_empresa', 'sector', 'industria', 'timestamp']

# Insertar los datos en la tabla 'empresas'
df_empresas.to_sql('empresas', con=engine, if_exists='append', index=False)
print("Datos insertados en la tabla 'empresas'")

### 2. Poblar la Tabla `precios_historicos` ###

# Cargar el CSV de precios históricos
df_precios = pd.read_csv('../Streamlit/Cotizaciones_historicas_SP_500.csv.csv')  # Reemplaza con la ruta al archivo real

# Seleccionar y renombrar columnas para que coincidan con la tabla `precios_historicos`
df_precios = df_precios[['Date', 'Open', 'Close', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits', 'Symbol']]
df_precios.columns = ['fecha', 'precio_apertura', 'precio_cierre', 'maximo', 'minimo', 'volumen', 'dividends', 'stock_splits', 'ticker']

# Mapear `ticker` de `precios_historicos` con `id_empresa` de `empresas`
df_empresas_db = pd.read_sql('SELECT id_empresa, ticker FROM empresas', con=engine)
df_precios = df_precios.merge(df_empresas_db, on='ticker', how='left')

# Eliminar la columna de `ticker` para dejar solo `id_empresa` y poblar la tabla `precios_historicos`
df_precios = df_precios[['id_empresa', 'fecha', 'precio_apertura', 'precio_cierre', 'maximo', 'minimo', 'volumen', 'dividends', 'stock_splits']]
df_precios.to_sql('precios_historicos', con=engine, if_exists='append', index=False)
print("Datos insertados en la tabla 'precios_historicos'")

### 3. Poblar la Tabla `indicadores_sp500` ###

# Calcular indicadores financieros a partir del archivo de precios históricos
# Usamos `df_precios` ya cargado y calculamos medias móviles y RSI

# Agrupamos por empresa y ordenamos por fecha
df_precios = df_precios.sort_values(by=['id_empresa', 'fecha'])
df_indicadores = pd.DataFrame()

# Función para calcular el RSI
def calculate_rsi(data, window=14):
    delta = data['precio_cierre'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Calcular indicadores por cada empresa
for id_empresa, group in df_precios.groupby('id_empresa'):
    group['media_movil'] = group['precio_cierre'].rolling(window=50).mean()  # Media Móvil de 50 días
    group['rsi'] = calculate_rsi(group)                                      # RSI de 14 días
    group['volatilidad'] = group['precio_cierre'].pct_change().rolling(window=30).std()  # Volatilidad de 30 días
    df_indicadores = pd.concat([df_indicadores, group])

# Seleccionar columnas necesarias y poblar la tabla `indicadores_sp500`
df_indicadores = df_indicadores[['id_empresa', 'fecha', 'media_movil', 'rsi', 'volatilidad']]
df_indicadores.to_sql('indicadores_sp500', con=engine, if_exists='append', index=False)
print("Datos insertados en la tabla 'indicadores_sp500'")

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'sql301.infinityfree.com' ([Errno 8] nodename nor servname provided, or not known)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
import pymysql

# Datos de conexión
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
user = 'admin'                       # Cambia por tu usuario de RDS
password = '11jablum11'                   # Cambia por tu contraseña de RDS
database = 'yfinance'              # Cambia por el nombre de tu base de datos
port = 3306                                  # Puerto predeterminado para MySQL

# Crear la conexión
try:
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        #database=database,
        port=port
    )
    print("Conexión exitosa a la base de datos")
# Crear un cursor y crear la base de datos
    cursor = connection.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS yfinance")  # Cambia 'pfb' al nombre que prefieras
    print("Base de datos 'yfinance' creada o ya existente")

except pymysql.MySQLError as e:
    print("Error al conectar o crear la base de datos:", e)

finally:
    # Cerrar la conexión
    if connection:
        connection.close()



Conexión exitosa a la base de datos
Base de datos 'yfinance' creada o ya existente


In [2]:
# Sentencia SQL para crear las tablas
create_tables_query = """
CREATE TABLE IF NOT EXISTS empresas_sp500 (
    id_empresa INT AUTO_INCREMENT PRIMARY KEY,
    simbolo VARCHAR(10) NOT NULL,
    nombre_empresa VARCHAR(100),
    sector VARCHAR(50),
    industria VARCHAR(100),
    capitalizacion_mercado FLOAT
);

CREATE TABLE IF NOT EXISTS precios_historicos (
    id_precio INT AUTO_INCREMENT PRIMARY KEY,
    id_empresa INT,
    fecha DATE,
    precio_apertura FLOAT,
    precio_cierre FLOAT,
    maximo FLOAT,
    minimo FLOAT,
    volumen BIGINT
);

CREATE TABLE IF NOT EXISTS portafolios_usuarios (
    id_portafolio INT AUTO_INCREMENT PRIMARY KEY,
    nombre_portafolio VARCHAR(100),
    descripcion TEXT
);

CREATE TABLE IF NOT EXISTS portafolio_empresas (
    id_portafolio_empresa INT AUTO_INCREMENT PRIMARY KEY,
    id_portafolio INT,
    id_empresa INT,
    cantidad_acciones FLOAT,
    precio_compra FLOAT
);

CREATE TABLE IF NOT EXISTS indicadores_sp500 (
    id_indicador INT AUTO_INCREMENT PRIMARY KEY,
    id_empresa INT,
    fecha DATE,
    media_movil FLOAT,
    rsi FLOAT,
    volatilidad FLOAT
);
"""

try:
    # Conectar a la base de datos MySQL
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database,
        port=port
    )
    print("Conexión exitosa a la base de datos")

    # Crear un cursor y ejecutar el script SQL para crear tablas
    cursor = connection.cursor()
    for statement in create_tables_query.split(";"):
        if statement.strip():
            cursor.execute(statement)
    connection.commit()  # Confirmar cambios en la base de datos
    print("Tablas creadas exitosamente")

except pymysql.MySQLError as e:
    print("Error al conectar o ejecutar la consulta SQL:", e)

finally:
    # Cerrar la conexión
    if connection:
        connection.close()

Conexión exitosa a la base de datos
Tablas creadas exitosamente


In [4]:
import pandas as pd

In [8]:
import numpy as np

KeyboardInterrupt: 

In [31]:
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
user = 'admin'                       # Cambia por tu usuario de RDS
password = '11jablum11'                   # Cambia por tu contraseña de RDS
database = 'yfinance'              # Cambia por el nombre de tu base de datos
port = 3306   

connection = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
cursor = connection.cursor()

# Consulta para obtener el precio máximo de Apple el 6 de enero de 2000
query = """
SELECT maximo 
FROM precios_historicos
WHERE id_empresa = 'AAPL' AND fecha = '2020-06-1';
"""

try:
    cursor.execute(query)
    resultado = cursor.fetchone()
    if resultado:
        print(f"El precio máximo de Apple el 6 de enero de 2000 fue: {resultado[0]}")
    else:
        print("No se encontraron datos para la consulta especificada.")
except Exception as e:
    print("Error al ejecutar la consulta:", e)
finally:
    # Cerrar la conexión a la base de datos
    cursor.close()
    connection.close()

El precio máximo de Apple el 6 de enero de 2000 fue: 110.26


In [19]:
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
user = 'admin'                       # Cambia por tu usuario de RDS
password = '11jablum11'                   # Cambia por tu contraseña de RDS
database = 'yfinance'              # Cambia por el nombre de tu base de datos
port = 3306 

connection = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
cursor = connection.cursor()

# Consulta para obtener la información de Apple desde la tabla empresas_sp500
query = """
SELECT * 
FROM empresas_sp500
WHERE simbolo = 'AAPL';
"""

try:
    cursor.execute(query)
    resultado = cursor.fetchone()
    if resultado:
        # Desplegar la información en formato clave-valor
        columnas = [desc[0] for desc in cursor.description]  # Obtener nombres de columnas
        info_apple = dict(zip(columnas, resultado))  # Crear diccionario con los resultados
        print("Información de Apple en la tabla empresas_sp500:")
        for clave, valor in info_apple.items():
            print(f"{clave}: {valor}")
    else:
        print("No se encontraron datos para la empresa Apple.")
except Exception as e:
    print("Error al ejecutar la consulta:", e)
finally:
    # Cerrar la conexión a la base de datos
    cursor.close()
    connection.close()

Información de Apple en la tabla empresas_sp500:
id_empresa: 100
simbolo: AAPL
nombre_empresa: Apple Inc.
sector: Technology
industria: Consumer Electronics
capitalizacion_mercado: 3518380000000.0


In [17]:
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'  # Cambia por tu endpoint de RDS
user = 'admin'                       # Cambia por tu usuario de RDS
password = '11jablum11'                   # Cambia por tu contraseña de RDS
database = 'yfinance'              # Cambia por el nombre de tu base de datos
port = 3306   

connection = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
cursor = connection.cursor()

cargar_precios_historicos()

Datos de precios_historicos cargados correctamente


In [33]:
import pymysql

# Datos de conexión
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def consultar_precio_cierre_simple(simbolo, fecha):
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta simple para verificar datos en precios_historicos
        query = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query, (simbolo, fecha))
        resultado = cursor.fetchone()

        # Verificar si se obtuvo un precio de cierre
        if resultado:
            precio_cierre = resultado[0]
            print(f"Precio de cierre para {simbolo} en la fecha {fecha}: {precio_cierre}")
        else:
            print(f"No se encontró precio de cierre para {simbolo} en la fecha {fecha}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        # Cerrar la conexión
        if connection:
            connection.close()

# Ejemplo de uso de la función
consultar_precio_cierre_simple("AAPL", "2020-06-01")  # Cambia "AAPL" y la fecha por los valores que necesitas verificar



Precio de cierre para AAPL en la fecha 2020-06-01: 108.881


In [34]:
import pymysql

# Datos de conexión
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def calcular_roi(simbolo, fecha_compra, fecha_venta):
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Consulta para el precio de compra (precio de cierre en la fecha de compra)
        query_compra = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_compra, (simbolo, fecha_compra))
        resultado_compra = cursor.fetchone()

        # Consulta para el precio de venta (precio de cierre en la fecha de venta)
        query_venta = """
        SELECT precio_cierre
        FROM precios_historicos
        WHERE id_empresa = %s AND fecha = %s
        """
        cursor.execute(query_venta, (simbolo, fecha_venta))
        resultado_venta = cursor.fetchone()

        # Verificar que se encontraron precios en ambas fechas
        if resultado_compra and resultado_venta:
            precio_compra = resultado_compra[0]
            precio_venta = resultado_venta[0]

            # Calcular ROI
            ganancia = precio_venta - precio_compra
            roi = (ganancia / precio_compra) * 100

            print(f"Empresa: {simbolo}")
            print(f"Precio de compra ({fecha_compra}): {precio_compra}")
            print(f"Precio de venta ({fecha_venta}): {precio_venta}")
            print(f"ROI entre {fecha_compra} y {fecha_venta}: {round(roi, 2)}%")
        else:
            if not resultado_compra:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de compra {fecha_compra}.")
            if not resultado_venta:
                print(f"No se encontró precio de cierre para {simbolo} en la fecha de venta {fecha_venta}.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        # Cerrar la conexión
        if connection:
            connection.close()

# Ejemplo de uso de la función
calcular_roi("AAPL", "2020-01-02", "2020-12-31")  # Cambia "AAPL" y las fechas por los valores que necesitas verificar


Empresa: AAPL
Precio de compra (2020-01-02): 123.615
Precio de venta (2020-12-31): 124.507
ROI entre 2020-01-02 y 2020-12-31: 0.72%


In [36]:
import pymysql
import pandas as pd

# Datos de conexión a la base de datos
host = 'pfb.cp2wsq8yih32.eu-north-1.rds.amazonaws.com'
user = 'admin'
password = '11jablum11'
database = 'yfinance'
port = 3306

def recrear_tabla_precios_historicos():
    """Elimina y recrea la tabla precios_historicos."""
    try:
        # Conectar a la base de datos
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Eliminar la tabla si existe
        cursor.execute("DROP TABLE IF EXISTS precios_historicos;")

        # Crear la tabla de nuevo
        create_table_query = """
        CREATE TABLE precios_historicos (
            id_precio INT AUTO_INCREMENT PRIMARY KEY,
            id_empresa INT,
            fecha DATE,
            precio_apertura FLOAT,
            precio_cierre FLOAT,
            maximo FLOAT,
            minimo FLOAT,
            volumen BIGINT
        );
        """
        cursor.execute(create_table_query)
        print("Tabla precios_historicos recreada correctamente.")

        # Confirmar cambios
        connection.commit()

    except pymysql.MySQLError as e:
        print(f"Error al recrear la tabla: {e}")

    finally:
        if connection:
            connection.close()

def cargar_datos_precios_historicos(filepath):
    """Carga los datos desde un archivo CSV y asigna el id_empresa correcto a cada registro."""
    # Leer archivo CSV con los datos de precios históricos
    precios_df = pd.read_csv(filepath)

    try:
        # Conectar a la base de datos para obtener el mapping de `Symbol` a `id_empresa`
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            port=port
        )
        cursor = connection.cursor()

        # Obtener el mapping de `simbolo` a `id_empresa` de `empresas_sp500`
        cursor.execute("SELECT simbolo, id_empresa FROM empresas_sp500")
        simbolo_id_mapping = {simbolo: id_empresa for simbolo, id_empresa in cursor.fetchall()}
        
        # Cerrar el cursor temporal
        cursor.close()
        
        # Agregar `id_empresa` en el DataFrame de precios usando el mapping
        precios_df['id_empresa'] = precios_df['Symbol'].map(simbolo_id_mapping)
        
        # Eliminar filas sin `id_empresa` (si no hubo coincidencias en el mapping)
        precios_df = precios_df.dropna(subset=['id_empresa'])
        precios_df['id_empresa'] = precios_df['id_empresa'].astype(int)  # Convertir id_empresa a entero

        # Reabrir el cursor para insertar los datos
        cursor = connection.cursor()

        # Insertar datos en `precios_historicos`
        for _, row in precios_df.iterrows():
            insert_query = """
            INSERT INTO precios_historicos (id_empresa, fecha, precio_apertura, precio_cierre, maximo, minimo, volumen)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, (
                row['id_empresa'], row['Date'], row['Open'], 
                row['Close'], row['High'], row['Low'], row['Volume']
            ))

        # Confirmar cambios
        connection.commit()
        print("Datos de precios históricos cargados correctamente.")

    except pymysql.MySQLError as e:
        print("Error al conectar o ejecutar la consulta SQL:", e)

    finally:
        if connection:
            connection.close()

# Uso de las funciones
recrear_tabla_precios_historicos()



Tabla precios_historicos recreada correctamente.


In [37]:
cargar_datos_precios_historicos('Cotizaciones_historicas_SP_500.csv')  

Datos de precios históricos cargados correctamente.
